In [93]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


In [120]:
df1=pd.read_csv('train fnews.csv')

In [121]:
df2=pd.read_csv('test fnews.csv')

In [122]:
df1.fillna('',inplace=True)

In [123]:
df2.fillna('',inplace=True)

In [124]:
df1

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [125]:
df1['comb']=df1['title']+' '+df1['author']

In [126]:
df1['comb']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: comb, Length: 20800, dtype: object

In [145]:
df1

,index,id,title,author,text,label,comb
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...,...,...
20795,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [128]:
voc_size=5000
ps=PorterStemmer()
import re

In [129]:
df2['comb']=df2['title']+' '+df2['author']

In [130]:
df1.reset_index(inplace=True)
corpus_train=[]
for i in range(len(df1)):
    news=re.sub('[^a-zA-Z]',' ',df1['comb'][i])
    news.lower()
    news=news.split()
    news=[ps.stem(word) for word in news if word not in stopwords.words('english')]
    news=' '.join(news)
    corpus_train.append(news)

In [131]:
df2.reset_index(inplace=True)
corpus_test=[]
for i in range(len(df2)):
    news=re.sub('[^a-zA-Z]',' ',df2['comb'][i])
    news.lower()
    news=news.split()
    news=[ps.stem(word) for word in news if word not in stopwords.words('english')]
    news=' '.join(news)
    corpus_test.append(news)

In [132]:
one_hot_train=[one_hot(sent,n=voc_size) for sent in corpus_train]

In [133]:
one_hot_test=[one_hot(sent,n=voc_size) for sent in corpus_test]

In [134]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [135]:
emb_train=pad_sequences(one_hot_train,maxlen=25)

In [136]:
emb_test=pad_sequences(one_hot_test,maxlen=25)

In [137]:
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding
from tensorflow.keras.models import Sequential

In [149]:
model=Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(LSTM(100))
model.add(Dropout(0.25))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))


In [150]:
model.compile( optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [151]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                6464      
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 262,929
Trainable params: 262,929
Non-trainable params: 0
________________________________________________

In [152]:
X_train=np.array(emb_train)
y_train=np.array(df1['label'])

In [153]:
model.fit(x=X_train,y=y_train,epochs=15,batch_size=64)

Epoch 1/15
325/325 [==============================] - 5s 12ms/step - loss: 0.3088 - accuracy: 0.8482
Epoch 2/15
325/325 [==============================] - 4s 12ms/step - loss: 0.0158 - accuracy: 0.9955
Epoch 3/15
325/325 [==============================] - 4s 12ms/step - loss: 0.0040 - accuracy: 0.9991
Epoch 4/15
325/325 [==============================] - 4s 12ms/step - loss: 0.0013 - accuracy: 0.9997
Epoch 5/15
325/325 [==============================] - 4s 12ms/step - loss: 8.0630e-04 - accuracy: 0.9998
Epoch 6/15
325/325 [==============================] - 5s 15ms/step - loss: 0.0013 - accuracy: 0.9996
Epoch 7/15
325/325 [==============================] - 8s 26ms/step - loss: 0.0023 - accuracy: 0.9993
Epoch 8/15
325/325 [==============================] - 8s 26ms/step - loss: 8.9429e-04 - accuracy: 0.9996
Epoch 9/15
325/325 [==============================] - 8s 25ms/step - loss: 1.5327e-04 - accuracy: 1.0000
Epoch 10/15
325/325 [==============================] - 8s 25ms/step - loss: 1.1

In [143]:
X_train.shape

(20800, 25)

In [147]:
y_train.shape

(20800,)

In [154]:
X_test = np.array(emb_test)

In [155]:
y_pred = model.predict_classes(X_test)

C:\Users\ANIKAIT\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [158]:
final_sub = pd.DataFrame()
final_sub['id']=df2['id']
final_sub['label'] = y_pred
final_sub.to_csv('final_sub.csv',index=False)